In [74]:
import database_connected_naver as dcn
from selenium import webdriver
import time
import random
import pandas as pd

datas = {}

#checkVirusArea

def scroll_down():  #마지막 element 까지 scroll_down 하는건듯
    last_ele = driver.find_element_by_css_selector('.list_basis > div > div:last-child') # 마지막 element를 찾는것
    driver.execute_script('arguments[0].scrollIntoView()', last_ele) #java script함수 (뭔얘기임?)
    return len(driver.find_elements_by_css_selector('.list_basis > div > div')) #java script 함수


def scroll_ready():
    g_cnt = len(driver.find_elements_by_css_selector('.list_basis > div > div')) #갯수를 센다 (div가 총 몇개인가)
    while g_cnt < 80: # 갯수 = 정해진 page size로 구성 or site에서 80으로 변경하거나
        g_cnt = scroll_down() #count가 80보다 클 때 까지 돈다. 
        time.sleep(random.randint(1, 2)) # 1 or 2 중 랜덤하게 가져와서 잠깐 멈추라 


def next_page():  # 다음페이지로 가는것
    try:
        next_page_button = driver.find_element_by_css_selector('a[class^="pagination_next__1ITTf"]')
        next_page_button.click()
    except:
        return False
    time.sleep(random.randint(1, 2))
    return True

driver = webdriver.Chrome('./chromedriver')
driver.get('https://search.shopping.naver.com/search/category?catId=50006656&frm=NVSHCAT&origQuery&pagingIndex=1&pagingSize=80&productSet=model&query&sort=rel&timestamp=&viewType=list') #pagingindex=1이면 1페이지에 멈춰있음. 이 페이지를 변
# 페이지 80페이지인지 꼭 확인하자!!! -> 80을 변경하려면 위에 g_cnt도 변경 필요 !
def db_collect():
    df = pd.DataFrame()
    node_list = driver.find_elements_by_css_selector('.list_basis > div > div')
    for node in node_list:
        a = node.find_element_by_css_selector('div[class^="basicList_price_area"]')
        driver.execute_script('arguments[0].scrollIntoView()', a)
        if '광고' not in a.text:
            t = node.find_element_by_css_selector('div[class^="basicList_title"] > a')
            prod_name = t.get_attribute('title')
            prod_name = prod_name.replace("'", "")
            thumb = ''
            try:
                thumb_tag = node.find_element_by_css_selector('a[class^="thumbnail_thumb"] > img')
                thumb = thumb_tag.get_attribute('src')
            except:
                print('thumb err', prod_name)
            price_temp = a.find_elements_by_css_selector('strong[class^="basicList_price"] > span > span')
            if len(price_temp) > 0:
                zzim = node.find_element_by_css_selector('button[class^="basicList_btn_zzim"]')
                parent_pcode = zzim.get_attribute('data-i')
                desc = node.find_element_by_css_selector('div[class^="basicList_depth"]').text
                desc += node.find_element_by_css_selector('div[class^="basicList_detail_box"]').text
                s1 = {'parent_pcode': parent_pcode,'product_name': prod_name, 'thumb': thumb,'product_desc': desc,'href': 'https://search.shopping.naver.com/catalog/'+parent_pcode }
                df = df.append(s1, ignore_index= True)
    df
#                dcn.push_prod_data_parent({ #dcn의 push_prod_data_parent에 넣어 
#                     'parent_pcode': parent_pcode,
#                     'product_name': prod_name,
#                     'thumb': thumb,
#                     'product_desc': desc,
#                     'href': 'https://search.shopping.naver.com/catalog/'+parent_pcode
#                })

# 강아지 : https://search.shopping.naver.com/search/category?catId=50006630&frm=NVSHMDL&origQuery&pagingIndex=1&pagingSize=80&productSet=model&query&sort=rel&timestamp=&viewType=list
# 고양이 : https://search.shopping.naver.com/search/category?catId=50006678&frm=NVSHMDL&origQuery&pagingIndex=1&pagingSize=80&productSet=model&query&sort=rel&timestamp=&viewType=list

# 강아지 영양제 : https://search.shopping.naver.com/search/category?catId=50006655&frm=NVSHCAT&origQuery&pagingIndex=1&pagingSize=40&productSet=model&query&sort=rel&timestamp=&viewType=list
# 강아지 유산균 : https://search.shopping.naver.com/search/category?catId=50006656&frm=NVSHCAT&origQuery&pagingIndex=1&pagingSize=40&productSet=model&query&sort=rel&timestamp=&viewType=list
# 고양이 영양제 : 
# 고양이 유산균 : 

In [72]:
go_stop = True
cnt = 1
while go_stop:
    print(cnt)
    if cnt == 1:
        scroll_ready()
    db_collect()
    scroll_down()
    go_stop = next_page() 
    cnt += 1

1


KeyboardInterrupt: 

In [ ]:
driver.quit()

In [75]:
db_collect()

print (df)

   기러기   참새
0  0.0  1.0


In [73]:
df

,기러기,참새
0,0.0,1.0
